# Local Model
- get data
- fix data -> deal with missing values
- train model 
- predict on request (first fill missing values?)



In [1]:
# Import all the useful libraries
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary as pds
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1500
import fancyimpute
from sklearn.preprocessing import StandardScaler


from sklearn import model_selection
# classifiers
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn import naive_bayes
from sklearn import svm
from sklearn.linear_model import LogisticRegression



#### necessary?



import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   


import matplotlib.pyplot as plt
%matplotlib inline

# ignore seaborn warnings
import warnings
warnings.filterwarnings("ignore")


Using TensorFlow backend.


In [2]:
# training data -> expected to be with all the listed features (IN ORDER -> like in the data we have). It is ok, if there are missing values

class LocalModel_old:
    
    # local model functions
    # train
    # predict 
   
    
    # initialize the local model with the training data
    
    target = 'AFclass'
    original_data = [] # the initial dataset
    data = [] # the cleaned data
    features = []
    original_test = [] # not modified test data
    test_set = [] # cleaned test data
    
    # models 
    sl = [] # logistic regression
    rf = [] # random forest
    knn = [] # knn
    nb = [] # naive bayes
    svc = [] # support vector classifier
    
    def __init__(self, data):
        self.data = data
        self.original_data = data
        
    def showData(self, lines = 5):
        print(self.data.head(lines))
        
    # remove unused features, convert categorical attributes to numerical ones
    def cleanData(self):
        print ("START CLEANING")
        self.data = self.data.drop('Soggetti', axis = 1)
        
        
        self.data = self.data.drop('PCneg', axis = 1)
        self.data = self.data.drop('IPG', axis = 1)
        # convert categorical variables into numerical 
        self.data['patsex'] = self.data['patsex'].map({'männlich' : 1, 'weiblich' : 0})
        self.data["AFclass"] = self.data["AFclass"].map({'persistierend (>7 Tage, EKV)' : 1, 'paroxysmal' : 0}) 
       # extract features
        self.features = self.data.columns[self.data.columns != self.target]
        print ("END CLEANING")
    
    def cleanDataTest(self):
        print ("START TEST CLEANING")
        self.test_set = self.test_set.drop('Soggetti', axis = 1)
        self.test_set = self.test_set.drop('PCneg', axis = 1)
        self.test_set = self.test_set.drop('IPG', axis = 1)
        # convert categorical variables into numerical 
        self.test_set['patsex'] = self.test_set['patsex'].map({'männlich' : 1, 'weiblich' : 0})
        print ("END TEST CLEANING")
    
    
    def recoverMissing(self):
        print ("START RecoverMissing VALUES")
        x_incomplete = self.data[self.features]
        y = self.data[self.target].as_matrix()
        # impute missing value with KNN strategy
        x_knn_a = fancyimpute.KNN(15).complete(x_incomplete)
        x_knn = pd.DataFrame(x_knn_a, columns = self.features)
        y_new = pd.DataFrame(y)
        y_new = y_new.rename(columns = {y_new.columns[0] : 'AFclass'})
        Data_KNN = pd.concat([x_knn,y_new], axis = 1)
        self.data = Data_KNN
        print ("END RecoverMissing VALUES")
    
    def recoverMissingTest(self):
        print ("START RecoverMissing VALUES TEST")   
        # impute missing value with KNN strategy
        x_knn_a = fancyimpute.KNN(15).complete(self.test_set)
        x_knn = pd.DataFrame(x_knn_a, columns = self.features)
        self.test_set = x_knn
        print ("END RecoverMissing VALUES TEST")
   

    # train the models
    def train(self):
        
        self.cleanData()
        self.recoverMissing()
        
        # use all  the data for training ? is it a good idea ? check questions file
        x = self.data[self.features]
        y = self.data[self.target]
        random_state = 11223344
        print ("START TRAINING")
        self.sl_train(x,y,random_state)
        self.rf_train(x,y,random_state)
        self.knn_train(x,y)
        self.nb_train(x,y)
        self.svc_train(x,y,random_state)
        print ("END TRAINING")
    
    def sl_train(self,x,y,random_state):
        print ("start sl training")
        self.sl = LogisticRegression(C=10e10, random_state=random_state);
        self.sl.fit(x,y);
        
        #y_p = sl.predict(test);
        print ("end sl training")

    def rf_train(self,x,y,random_state):
        print ("start rf training")
        self.rf = RandomForestClassifier(n_estimators=40, max_depth=None, min_samples_split=2, random_state=random_state)
        self.rf.fit(x,y);
        print ("features importance")
        print (self.rf.feature_importances_)
        
        #Y_P_bal_rf_1 = rf_cm.predict(X_bal_1);
        print ("end rf training")
            
    def knn_train(self,x,y):
        print ("start knn training")
        self.knn = neighbors.KNeighborsClassifier(20, weights='distance')
        self.knn.fit(x,y);
        #Y_P_bal_knn_1 = knn_cm.predict(X_bal_1);
        print ("end knn training")

    def nb_train(self,x,y):
        print ("start nb training")
        self.nb = naive_bayes.GaussianNB();
        self.nb.fit(x,y);
        #Y_P_bal_nb_1 = nb_cm.predict(X_bal_1);
        print ("end nb training")
    
    def svc_train (self,x,y,random_state):
        print ("start svm training")
        #self.svc = svm.SVC(kernel='linear', C = 1.0)
        self.svc = svm.SVC(kernel='rbf',C=1, probability=True, random_state=random_state)
        self.svc.fit(x,y)
        #(clf.predict([0.58,0.76]))
        print ("end svm training")
        

    def predict(self, x):
        print ("START PREDICTION")
        
        # check if x has null values -> if so ? impute missings ?
        self.test_set = x
        self.cleanDataTest()
        self.recoverMissingTest()
        
        sl_y_p = self.sl.predict(self.test_set)
        rf_y_p = self.rf.predict(self.test_set)
        print ("decision path -> random forest")
        print (self.rf.decision_path(self.test_set))
        knn_y_p = self.knn.predict(self.test_set)
        nb_y_p = self.nb.predict(self.test_set)
        svc_y_p = self.svc.predict(self.test_set)
        
        # aggregate predictions -> MODE
        prediction = pd.DataFrame({"sl": sl_y_p, "rf": rf_y_p, "knn": knn_y_p, "nb" : nb_y_p, "svc" : svc_y_p})
        prediction['MODE'] = prediction.mode(axis= 1)
        print ("END PREDICTION")
        
        return prediction['MODE']

In [3]:
d = pd.read_pickle('../data/data.pickle')

In [4]:
# TEST

Data_pers = d[d['AFclass'] == 'persistierend (>7 Tage, EKV)']
Data_paro = d[d['AFclass'] == 'paroxysmal']
Data_paro_1 = Data_paro.sample(n=400, random_state = 1234, replace = False)

balanced_1 = pd.concat([Data_pers,Data_paro_1])

X_bal_1 = balanced_1[balanced_1.columns[balanced_1.columns != "AFclass"]]
Y_bal_1 = balanced_1["AFclass"]


train_x, test_x_p, train_y, test_y = model_selection.train_test_split(X_bal_1, Y_bal_1, test_size=0.1, random_state=1242)

#train_x.shape

#trainData = pd.concat([train_x,train_y])
train_y = pd.DataFrame(train_y)
trainData = pd.concat([train_x,train_y], axis = 1)
trainData.head()

test_x_p.head()

,Soggetti,RR,QTm_old,sbjBeatConsidered,numRRaveraged,QR,QTn,QRS,IPG,PQ,PCpos,PCneg,PCdur,PCfwhm,PCarea,P_LoffEon,Pdur16mean,Pdur16median,Pdur16min,Pdur16max,Pdur16std,Pdur16iqr,Pdur_ECG_I,Pdur_ECG_II,Pdur_ECG_III,Pdur_ECG_V1,Pdur_ECG_V2,Pdur_ECG_V3,Pdur_ECG_V3R,Pdur_ECG_V4,Pdur_ECG_V4R,Pdur_ECG_V5,Pdur_ECG_V6,Pdur_ECG_V8,Pdur_ECG_V9,Pdur_ECG_aVF,Pdur_ECG_aVL,Pdur_ECG_aVR,Parea_ECG_I,Parea_ECG_II,Parea_ECG_III,Parea_ECG_V1,Parea_ECG_V2,Parea_ECG_V3,Parea_ECG_V3R,Parea_ECG_V4,Parea_ECG_V4R,Parea_ECG_V5,Parea_ECG_V6,Parea_ECG_V8,Parea_ECG_V9,Parea_ECG_aVF,Parea_ECG_aVL,Parea_ECG_aVR,Pfwhm_ECG_I,Pfwhm_ECG_II,Pfwhm_ECG_III,Pfwhm_ECG_V1,Pfwhm_ECG_V2,Pfwhm_ECG_V3,Pfwhm_ECG_V3R,Pfwhm_ECG_V4,Pfwhm_ECG_V4R,Pfwhm_ECG_V5,Pfwhm_ECG_V6,Pfwhm_ECG_V8,Pfwhm_ECG_V9,Pfwhm_ECG_aVF,Pfwhm_ECG_aVL,Pfwhm_ECG_aVR,patsex,Age
290,14_0008,1231.871681,431.0,232,226,38.0,388.324270,74.0,0,158.0,257.6910,NaN,128.0,60.02407,16394.320,152.0,129.6875,132.5,113.0,143.0,8.646531,14.0,120.0,137.0,143.0,138.0,134.0,130.0,121.0,131.0,136.0,131.0,135.0,116.0,122.0,134.0,113.0,134.0,6041.926,6999.822,2027.7160,3477.860,3276.1160,5200.566,2908.286,5538.657,2375.971,5370.082,5016.995,2309.9570,1317.4560,3564.955,2972.2530,6547.391,56.07841,63.00198,NaN,30.00344,38.19406,40.02066,30.29496,51.00406,32.00005,55.00683,62.00148,43.02146,34.00534,63.00161,51.00298,62.00538,männlich,76
48,01_2515,985.834123,442.0,254,211,22.0,445.164318,63.0,0,184.0,255.7596,NaN,125.0,62.00131,16809.310,136.0,123.5000,123.5,116.0,132.0,4.604346,7.0,116.0,126.0,130.0,120.0,123.0,132.0,117.0,127.0,120.0,122.0,121.0,124.0,119.0,127.0,128.0,124.0,4027.367,8851.807,4526.3600,3783.728,924.7281,1190.772,3043.634,2766.957,2650.744,3915.786,4353.544,1203.7370,3352.9430,6654.358,1023.4170,6471.130,51.00261,70.00912,60.002690,50.00336,37.00084,NaN,46.06335,63.00037,44.00155,67.00427,62.00032,43.00067,40.01243,71.00185,NaN,63.00076,männlich,60
486,11_0139,923.526316,366.0,324,323,44.0,380.852182,81.0,0,189.0,240.2040,NaN,80.0,42.21366,10721.090,88.0,77.6875,78.0,70.0,85.0,5.055937,7.5,71.0,85.0,78.0,77.0,71.0,78.0,77.0,77.0,79.0,71.0,70.0,78.0,85.0,84.0,78.0,84.0,2076.926,3105.727,943.6826,2953.677,3961.2170,4501.361,3249.488,4132.521,2275.926,3275.238,2262.725,642.9807,938.5418,1827.491,636.3765,2521.194,23.12834,42.00917,9.096227,38.00850,42.03222,42.06930,40.00630,42.00581,45.00230,42.02562,38.05697,20.79247,16.54447,53.00960,19.00562,29.01267,männlich,60
46,01_2507,907.091185,429.0,330,329,42.0,450.434678,94.0,0,210.0,243.4206,-11.1445,159.0,45.00105,12445.320,186.0,154.0625,157.5,125.0,182.0,16.782800,20.5,148.0,127.0,141.0,157.0,125.0,128.0,182.0,156.0,158.0,163.0,169.0,160.0,162.0,167.0,173.0,149.0,5359.116,3487.825,1293.4350,5691.185,5870.5750,4909.660,4236.073,3858.455,3079.584,4103.849,4241.803,3411.5560,3146.6390,4905.731,2016.8670,6237.559,54.00025,50.01936,30.016690,70.00323,41.14713,45.00491,67.00006,47.00000,43.00135,57.00009,108.00190,81.00204,70.00186,66.00296,71.00043,84.00625,männlich,48
375,10_0010,1027.147059,378.0,277,272,44.0,372.971362,82.0,0,173.0,109.0741,-69.0900,132.0,NaN,6463.218,164.0,129.5000,128.5,111.0,151.0,13.125040,21.5,145.0,115.0,125.0,123.0,150.0,131.0,126.0,133.0,122.0,151.0,114.0,137.0,143.0,113.0,111.0,133.0,3922.767,1794.308,1271.2550,2311.861,2771.5950,2614.647,2302.935,2269.173,1929.329,2133.636,1391.586,1639.3270,1475.2450,1426.139,1462.8890,3786.315,85.00373,83.00005,NaN,47.00002,NaN,49.03215,47.00508,44.00050,48.00081,47.00836,40.00693,46.00161,42.01393,46.00000,62.00243,97.00637,männlich,71


In [5]:
lm = LocalModel_old(trainData)


In [6]:
lm.train()

START CLEANING
END CLEANING
START RecoverMissing VALUES
Imputing row 1/676 with 1 missing, elapsed time: 0.249
Imputing row 101/676 with 0 missing, elapsed time: 0.414
Imputing row 201/676 with 2 missing, elapsed time: 0.427
Imputing row 301/676 with 60 missing, elapsed time: 0.444
Imputing row 401/676 with 4 missing, elapsed time: 0.460
Imputing row 501/676 with 0 missing, elapsed time: 0.474
Imputing row 601/676 with 60 missing, elapsed time: 0.485
END RecoverMissing VALUES
START TRAINING
start sl training
end sl training
start rf training
features importance
[ 0.01472633  0.03019805  0.0116273   0.01874787  0.01294035  0.03234499
  0.02189451  0.01033661  0.01486354  0.01044863  0.01232853  0.01438024
  0.00821339  0.01116516  0.00943228  0.00769729  0.01053319  0.01588951
  0.01424073  0.01007211  0.01202641  0.01637388  0.00968654  0.00614802
  0.01402247  0.00830804  0.01233819  0.00779838  0.01252219  0.00839539
  0.00980628  0.01413308  0.00826366  0.00777406  0.0139824   0.023

In [7]:
predicted_class = lm.predict(test_x_p)
predicted_y = predicted_class.ravel()

START PREDICTION
START TEST CLEANING
END TEST CLEANING
START RecoverMissing VALUES TEST
Imputing row 1/76 with 1 missing, elapsed time: 0.007
END RecoverMissing VALUES TEST
decision path -> random forest
(<76x8214 sparse matrix of type '<class 'numpy.int64'>'
	with 30414 stored elements in Compressed Sparse Row format>, array([   0,  213,  432,  647,  844, 1037, 1238, 1417, 1652, 1847, 2066,
       2279, 2486, 2687, 2890, 3097, 3300, 3509, 3722, 3919, 4142, 4345,
       4546, 4761, 4954, 5173, 5392, 5587, 5778, 5975, 6172, 6385, 6572,
       6767, 6962, 7179, 7394, 7595, 7786, 8007, 8214]))
END PREDICTION


In [8]:
test_y = test_y.map({'persistierend (>7 Tage, EKV)' : 1, 'paroxysmal' : 0}).ravel()

In [9]:
test_y

array([1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0])

In [10]:
countPerTrue = 0
countParTrue = 0
countPerFalse = 0
countParFalse = 0
for i in range(test_y.size):
    if (test_y[i] == predicted_y[i]) and (test_y[i]== 1):
        countPerTrue += 1
    if (test_y[i] == predicted_y[i]) and (test_y[i]== 0):
        countParTrue += 1
    if (test_y[i] != predicted_y[i]) and (test_y[i]== 1):
        countParFalse += 1
    if (test_y[i] != predicted_y[i]) and (test_y[i]== 0):
        countPerFalse += 1
    #print (Y_bal_1_array[i],Y_P_bal_1[i])
print ("FOR logistic regression")
print ("result true pers : ", countPerTrue)
print ("result true paro : ", countParTrue)
print ("result false pers : ", countPerFalse)
print ("result false paro : ", countParFalse)
print ("ACCURACY : ", (countPerTrue + countParTrue)/test_y.size)
print ("PRECISION : ", countPerTrue/ (countPerTrue+countPerFalse))
print ("RECALL : ", countPerTrue/ (countPerTrue+countParFalse))

FOR logistic regression
result true pers :  8
result true paro :  35
result false pers :  14
result false paro :  19
ACCURACY :  0.5657894736842105
PRECISION :  0.36363636363636365
RECALL :  0.2962962962962963



## test -> less feature in the trainig set than in the test set

## FAILD -> the test data cannot have more features that the train data


## test -> less feature in the test set than in the train set


## FAILD -> the test data cannot have less features that the train data


In [11]:
from LocalModel import LocalModel

In [12]:
g = LocalModel(data = trainData, target_name = "AFclass", model_name = "nb1")
g.train()
test = test_x_p.copy()
predicted_class = g.predict(test = test)
predicted = predicted_class["prediction"]
#predicted_class = g.predict(test_x_p)
predicted_y = predicted.ravel()
#test_y = test_y.map({'persistierend (>7 Tage, EKV)' : 1, 'paroxysmal' : 0}).ravel()


begin fixing dataset
START CLEANING
END CLEANING
START RecoverMissing VALUES
[MICE] Completing matrix with shape (676, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.004
[MICE] Starting imputation round 2/110, elapsed time 0.127
[MICE] Starting imputation round 3/110, elapsed time 0.160
[MICE] Starting imputation round 4/110, elapsed time 0.193
[MICE] Starting imputation round 5/110, elapsed time 0.229
[MICE] Starting imputation round 6/110, elapsed time 0.262
[MICE] Starting imputation round 7/110, elapsed time 0.295
[MICE] Starting imputation round 8/110, elapsed time 0.327
[MICE] Starting imputation round 9/110, elapsed time 0.362
[MICE] Starting imputation round 10/110, elapsed time 0.397
[MICE] Starting imputation round 11/110, elapsed time 0.437
[MICE] Starting imputation round 12/110, elapsed time 0.482
[MICE] Starting imputation round 13/110, elapsed time 0.522
[MICE] Starting imputation round 14/110, elapsed time 0.558
[MICE] Starting imputation round 15/110, elaps

[MICE] Starting imputation round 25/110, elapsed time 0.840
[MICE] Starting imputation round 26/110, elapsed time 0.877
[MICE] Starting imputation round 27/110, elapsed time 0.913
[MICE] Starting imputation round 28/110, elapsed time 0.950
[MICE] Starting imputation round 29/110, elapsed time 0.987
[MICE] Starting imputation round 30/110, elapsed time 1.022
[MICE] Starting imputation round 31/110, elapsed time 1.055
[MICE] Starting imputation round 32/110, elapsed time 1.094
[MICE] Starting imputation round 33/110, elapsed time 1.130
[MICE] Starting imputation round 34/110, elapsed time 1.165
[MICE] Starting imputation round 35/110, elapsed time 1.198
[MICE] Starting imputation round 36/110, elapsed time 1.231
[MICE] Starting imputation round 37/110, elapsed time 1.269
[MICE] Starting imputation round 38/110, elapsed time 1.304
[MICE] Starting imputation round 39/110, elapsed time 1.340
[MICE] Starting imputation round 40/110, elapsed time 1.375
[MICE] Starting imputation round 41/110,

In [13]:
countPerTrue = 0
countParTrue = 0
countPerFalse = 0
countParFalse = 0
for i in range(test_y.size):
    if (test_y[i] == predicted_y[i]) and (test_y[i]== 1):
        countPerTrue += 1
    if (test_y[i] == predicted_y[i]) and (test_y[i]== 0):
        countParTrue += 1
    if (test_y[i] != predicted_y[i]) and (test_y[i]== 1):
        countParFalse += 1
    if (test_y[i] != predicted_y[i]) and (test_y[i]== 0):
        countPerFalse += 1
    #print (Y_bal_1_array[i],Y_P_bal_1[i])
print ("FOR logistic regression")
print ("result true pers : ", countPerTrue)
print ("result true paro : ", countParTrue)
print ("result false pers : ", countPerFalse)
print ("result false paro : ", countParFalse)
print ("ACCURACY : ", (countPerTrue + countParTrue)/test_y.size)
print ("PRECISION : ", countPerTrue/ (countPerTrue+countPerFalse))
print ("RECALL : ", countPerTrue/ (countPerTrue+countParFalse))

FOR logistic regression
result true pers :  15
result true paro :  26
result false pers :  23
result false paro :  12
ACCURACY :  0.5394736842105263
PRECISION :  0.39473684210526316
RECALL :  0.5555555555555556


In [14]:
predicted_class


,RR,QTm_old,sbjBeatConsidered,numRRaveraged,QR,QTn,QRS,PQ,PCpos,PCdur,PCfwhm,PCarea,P_LoffEon,Pdur16mean,Pdur16median,Pdur16min,Pdur16max,Pdur16std,Pdur16iqr,Pdur_ECG_I,Pdur_ECG_II,Pdur_ECG_III,Pdur_ECG_V1,Pdur_ECG_V2,Pdur_ECG_V3,Pdur_ECG_V3R,Pdur_ECG_V4,Pdur_ECG_V4R,Pdur_ECG_V5,Pdur_ECG_V6,Pdur_ECG_V8,Pdur_ECG_V9,Pdur_ECG_aVF,Pdur_ECG_aVL,Pdur_ECG_aVR,Parea_ECG_I,Parea_ECG_II,Parea_ECG_III,Parea_ECG_V1,Parea_ECG_V2,Parea_ECG_V3,Parea_ECG_V3R,Parea_ECG_V4,Parea_ECG_V4R,Parea_ECG_V5,Parea_ECG_V6,Parea_ECG_V8,Parea_ECG_V9,Parea_ECG_aVF,Parea_ECG_aVL,Parea_ECG_aVR,Pfwhm_ECG_I,Pfwhm_ECG_II,Pfwhm_ECG_III,Pfwhm_ECG_V1,Pfwhm_ECG_V2,Pfwhm_ECG_V3,Pfwhm_ECG_V3R,Pfwhm_ECG_V4,Pfwhm_ECG_V4R,Pfwhm_ECG_V5,Pfwhm_ECG_V6,Pfwhm_ECG_V8,Pfwhm_ECG_V9,Pfwhm_ECG_aVF,Pfwhm_ECG_aVL,Pfwhm_ECG_aVR,patsex,Age,prediction,predict_proba_zero,predict_proba_uno,predict_log_proba_zero,predict_log_proba_uno
0,1231.871681,431.0,232.0,226.0,38.0,388.324270,74.000000,158.000000,257.691000,128.00000,60.024070,16394.320000,152.000000,129.687500,132.500000,113.000000,143.000000,8.646531,14.000000,120.000000,137.000000,143.000000,138.000000,134.000000,130.000000,121.000000,131.000000,136.000000,131.000000,135.000000,116.000000,122.000000,134.000000,113.000000,134.000000,6041.926000,6999.822000,2027.716000,3477.860000,3276.11600,5200.566000,2908.286000,5538.657000,2375.971000,5370.082000,5016.995000,2309.957000,1317.456000,3564.955000,2972.253000,6547.391000,56.078410,63.001980,52.726501,30.003440,38.194060,40.020660,30.294960,51.004060,32.000050,55.006830,62.001480,43.021460,34.005340,63.001610,51.002980,62.005380,1.0,76.0,1,0.444399,5.556008e-01,-8.110319e-01,-0.587705
1,985.834123,442.0,254.0,211.0,22.0,445.164318,63.000000,184.000000,255.759600,125.00000,62.001310,16809.310000,136.000000,123.500000,123.500000,116.000000,132.000000,4.604346,7.000000,116.000000,126.000000,130.000000,120.000000,123.000000,132.000000,117.000000,127.000000,120.000000,122.000000,121.000000,124.000000,119.000000,127.000000,128.000000,124.000000,4027.367000,8851.807000,4526.360000,3783.728000,924.72810,1190.772000,3043.634000,2766.957000,2650.744000,3915.786000,4353.544000,1203.737000,3352.943000,6654.358000,1023.417000,6471.130000,51.002610,70.009120,60.002690,50.003360,37.000840,56.436871,46.063350,63.000370,44.001550,67.004270,62.000320,43.000670,40.012430,71.001850,48.144267,63.000760,1.0,60.0,0,0.909547,9.045259e-02,-9.480816e-02,-2.402929
2,923.526316,366.0,324.0,323.0,44.0,380.852182,81.000000,189.000000,240.204000,80.00000,42.213660,10721.090000,88.000000,77.687500,78.000000,70.000000,85.000000,5.055937,7.500000,71.000000,85.000000,78.000000,77.000000,71.000000,78.000000,77.000000,77.000000,79.000000,71.000000,70.000000,78.000000,85.000000,84.000000,78.000000,84.000000,2076.926000,3105.727000,943.682600,2953.677000,3961.21700,4501.361000,3249.488000,4132.521000,2275.926000,3275.238000,2262.725000,642.980700,938.541800,1827.491000,636.376500,2521.194000,23.128340,42.009170,9.096227,38.008500,42.032220,42.069300,40.006300,42.005810,45.002300,42.025620,38.056970,20.792470,16.544470,53.009600,19.005620,29.012670,1.0,60.0,1,0.003005,9.969953e-01,-5.807585e+00,-0.003009
3,907.091185,429.0,330.0,329.0,42.0,450.434678,94.000000,210.000000,243.420600,159.00000,45.001050,12445.320000,186.000000,154.062500,157.500000,125.000000,182.000000,16.782800,20.500000,148.000000,127.000000,141.000000,157.000000,125.000000,128.000000,182.000000,156.000000,158.000000,163.000000,169.000000,160.000000,162.000000,167.000000,173.000000,149.000000,5359.116000,3487.825000,1293.435000,5691.185000,5870.57500,4909.660000,4236.073000,3858.455000,3079.584000,4103.849000,4241.803000,3411.556000,3146.639000,4905.731000,2016.867000,6237.559000,54.000250,50.019360,30.016690,70.003230,41.147130,45.004910,67.000060,47.000000,43.001350,57.000090,108.001900,81.002040,70.001860,66.002960,71.000430,84.006250,1.0,48.0,1,0.000162,9.998379e-01,-8.727285e+00,-0.000162
4,1027.147059,378.0,277.0,272.0,44.0,372.971362,82.000000,173.

In [15]:
test_x_p.head()

,Soggetti,RR,QTm_old,sbjBeatConsidered,numRRaveraged,QR,QTn,QRS,IPG,PQ,PCpos,PCneg,PCdur,PCfwhm,PCarea,P_LoffEon,Pdur16mean,Pdur16median,Pdur16min,Pdur16max,Pdur16std,Pdur16iqr,Pdur_ECG_I,Pdur_ECG_II,Pdur_ECG_III,Pdur_ECG_V1,Pdur_ECG_V2,Pdur_ECG_V3,Pdur_ECG_V3R,Pdur_ECG_V4,Pdur_ECG_V4R,Pdur_ECG_V5,Pdur_ECG_V6,Pdur_ECG_V8,Pdur_ECG_V9,Pdur_ECG_aVF,Pdur_ECG_aVL,Pdur_ECG_aVR,Parea_ECG_I,Parea_ECG_II,Parea_ECG_III,Parea_ECG_V1,Parea_ECG_V2,Parea_ECG_V3,Parea_ECG_V3R,Parea_ECG_V4,Parea_ECG_V4R,Parea_ECG_V5,Parea_ECG_V6,Parea_ECG_V8,Parea_ECG_V9,Parea_ECG_aVF,Parea_ECG_aVL,Parea_ECG_aVR,Pfwhm_ECG_I,Pfwhm_ECG_II,Pfwhm_ECG_III,Pfwhm_ECG_V1,Pfwhm_ECG_V2,Pfwhm_ECG_V3,Pfwhm_ECG_V3R,Pfwhm_ECG_V4,Pfwhm_ECG_V4R,Pfwhm_ECG_V5,Pfwhm_ECG_V6,Pfwhm_ECG_V8,Pfwhm_ECG_V9,Pfwhm_ECG_aVF,Pfwhm_ECG_aVL,Pfwhm_ECG_aVR,patsex,Age
290,14_0008,1231.871681,431.0,232,226,38.0,388.324270,74.0,0,158.0,257.6910,NaN,128.0,60.02407,16394.320,152.0,129.6875,132.5,113.0,143.0,8.646531,14.0,120.0,137.0,143.0,138.0,134.0,130.0,121.0,131.0,136.0,131.0,135.0,116.0,122.0,134.0,113.0,134.0,6041.926,6999.822,2027.7160,3477.860,3276.1160,5200.566,2908.286,5538.657,2375.971,5370.082,5016.995,2309.9570,1317.4560,3564.955,2972.2530,6547.391,56.07841,63.00198,NaN,30.00344,38.19406,40.02066,30.29496,51.00406,32.00005,55.00683,62.00148,43.02146,34.00534,63.00161,51.00298,62.00538,männlich,76
48,01_2515,985.834123,442.0,254,211,22.0,445.164318,63.0,0,184.0,255.7596,NaN,125.0,62.00131,16809.310,136.0,123.5000,123.5,116.0,132.0,4.604346,7.0,116.0,126.0,130.0,120.0,123.0,132.0,117.0,127.0,120.0,122.0,121.0,124.0,119.0,127.0,128.0,124.0,4027.367,8851.807,4526.3600,3783.728,924.7281,1190.772,3043.634,2766.957,2650.744,3915.786,4353.544,1203.7370,3352.9430,6654.358,1023.4170,6471.130,51.00261,70.00912,60.002690,50.00336,37.00084,NaN,46.06335,63.00037,44.00155,67.00427,62.00032,43.00067,40.01243,71.00185,NaN,63.00076,männlich,60
486,11_0139,923.526316,366.0,324,323,44.0,380.852182,81.0,0,189.0,240.2040,NaN,80.0,42.21366,10721.090,88.0,77.6875,78.0,70.0,85.0,5.055937,7.5,71.0,85.0,78.0,77.0,71.0,78.0,77.0,77.0,79.0,71.0,70.0,78.0,85.0,84.0,78.0,84.0,2076.926,3105.727,943.6826,2953.677,3961.2170,4501.361,3249.488,4132.521,2275.926,3275.238,2262.725,642.9807,938.5418,1827.491,636.3765,2521.194,23.12834,42.00917,9.096227,38.00850,42.03222,42.06930,40.00630,42.00581,45.00230,42.02562,38.05697,20.79247,16.54447,53.00960,19.00562,29.01267,männlich,60
46,01_2507,907.091185,429.0,330,329,42.0,450.434678,94.0,0,210.0,243.4206,-11.1445,159.0,45.00105,12445.320,186.0,154.0625,157.5,125.0,182.0,16.782800,20.5,148.0,127.0,141.0,157.0,125.0,128.0,182.0,156.0,158.0,163.0,169.0,160.0,162.0,167.0,173.0,149.0,5359.116,3487.825,1293.4350,5691.185,5870.5750,4909.660,4236.073,3858.455,3079.584,4103.849,4241.803,3411.5560,3146.6390,4905.731,2016.8670,6237.559,54.00025,50.01936,30.016690,70.00323,41.14713,45.00491,67.00006,47.00000,43.00135,57.00009,108.00190,81.00204,70.00186,66.00296,71.00043,84.00625,männlich,48
375,10_0010,1027.147059,378.0,277,272,44.0,372.971362,82.0,0,173.0,109.0741,-69.0900,132.0,NaN,6463.218,164.0,129.5000,128.5,111.0,151.0,13.125040,21.5,145.0,115.0,125.0,123.0,150.0,131.0,126.0,133.0,122.0,151.0,114.0,137.0,143.0,113.0,111.0,133.0,3922.767,1794.308,1271.2550,2311.861,2771.5950,2614.647,2302.935,2269.173,1929.329,2133.636,1391.586,1639.3270,1475.2450,1426.139,1462.8890,3786.315,85.00373,83.00005,NaN,47.00002,NaN,49.03215,47.00508,44.00050,48.00081,47.00836,40.00693,46.00161,42.01393,46.00000,62.00243,97.00637,männlich,71


In [16]:
# crossvalidation test
g.crossValidation(all_models = 1)

begin cross validation for all models
0.023  is complete 	
0.047  is complete 	
0.07  is complete 	
0.093  is complete 	
0.116  is complete 	
0.14  is complete 	
0.163  is complete 	
0.186  is complete 	
0.209  is complete 	
0.233  is complete 	
0.256  is complete 	
0.279  is complete 	
0.302  is complete 	
0.326  is complete 	
0.349  is complete 	
0.372  is complete 	
0.395  is complete 	
0.419  is complete 	
0.442  is complete 	
0.465  is complete 	
0.488  is complete 	
0.512  is complete 	
0.535  is complete 	
0.558  is complete 	
0.581  is complete 	
0.605  is complete 	
0.628  is complete 	
0.651  is complete 	
0.674  is complete 	
0.698  is complete 	
0.721  is complete 	
0.744  is complete 	
0.767  is complete 	
0.791  is complete 	
0.814  is complete 	
0.837  is complete 	
0.86  is complete 	
0.884  is complete 	
0.907  is complete 	
0.93  is complete 	
0.953  is complete 	
0.977  is complete 	
1.0  is complete 	
end cross validation


,0,1,2
0,knn4,62.40,3.64
1,gb4,61.71,6.99
2,gb3,60.55,7.84
3,knn5,60.27,5.00
4,ada4,59.72,6.44
5,et3,59.72,7.38
6,bag6,59.70,5.86
7,ada2,59.57,6.20
8,ada8,59.56,4.96
9,bag10,59.29,7.87


# local model .py TESTING

In [11]:
# Import all the useful libraries
import numpy as np
import pandas as pd
import fancyimpute
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold



from sklearn.ensemble import AdaBoostClassifier # PROBABILITY
from sklearn.tree import DecisionTreeClassifier # PROBABILITY
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier # PROBABILITY
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier # PROBABILITY
from sklearn.linear_model import LogisticRegression # PROBABILITY
from sklearn.naive_bayes import GaussianNB # PROBABILITY
from sklearn.ensemble import ExtraTreesClassifier # PROBABILITY
from sklearn.neighbors import KNeighborsClassifier # PROBABILITY
from sklearn.ensemble import BaggingClassifier # PROBABILITY

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import TomekLinks


# training data -> expected to be with all the listed features (IN ORDER -> like in the data we have). It is ok, if there are missing values

class LocalModel:
    
    # local model functions
    # train
    # predict 
   
    
    # initialize the local model with the training data
    '''
    target = 'AFclass'
    original_data = [] # the initial dataset
    data = [] # the cleaned data
    features = []
    original_test = [] # not modified test data
    test_set = [] # cleaned test data
    
    # models 
    sl = [] # logistic regression
    rf = [] # random forest
    knn = [] # knn
    nb = [] # naive bayes
    svc = [] # support vector classifier
    '''
    def __init__(self, data, target_name, model_name = "dt4",random_state = 12345678, imputation_strategy = 'mice',balance_strategy = 'SMOTE'):
        # we train the model with all the available data 
        self.target_name = target_name ## it the name of the target column
        self.target = None ## it is the target vector
        self.data = data ## it is the complete dataset -> will be modified
        self.original_data = data ## store a copy of the original data -> never modified
        self.X = None ## it is the data except the target
        self.features = None ## available features
        # for cross-validation 
        self.cv_x = None # data -> in principle equal to self.X
        self.cv_y = None # target -> in principle equal to self.target
        self.random_state = random_state # random state -> fixed for testing
        self.selected_model_name = 'dt4' # name of the model -> default fixed
        self.selected_model = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=15, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=self.random_state, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)  ## default model
        self.models = [] ## list of all the available models
        self.chosen_model(model_name) # select the chosen model -> otherwise use the default one
        self.fixDataset(imputation_strategy = imputation_strategy, balance_strategy = balance_strategy) ## fix data set before training -> clean data (remove unused columns, convert categotical attributes into numerical), recover missing values (use a strategy to impute the missing values), balance the data set

    # initiate the models 
    def models_definition(self,random_state):
        
        ## here we can tune the paramenters of the models
       

        self.models.append(("ada1",AdaBoostClassifier(DecisionTreeClassifier(max_depth=1, random_state = self.random_state),algorithm="SAMME", n_estimators=200)))
        self.models.append(("ada2",AdaBoostClassifier(DecisionTreeClassifier(max_depth=3, random_state = self.random_state),algorithm="SAMME", n_estimators=200)))
        self.models.append(("ada3",AdaBoostClassifier(DecisionTreeClassifier(max_depth=5, random_state = self.random_state),algorithm="SAMME", n_estimators=100)))
        self.models.append(("ada4",AdaBoostClassifier(DecisionTreeClassifier(max_depth=10, random_state = self.random_state),algorithm="SAMME", n_estimators=300)))
        self.models.append(("ada5",AdaBoostClassifier(DecisionTreeClassifier(max_depth=20, random_state = self.random_state),algorithm="SAMME", n_estimators=100)))
        self.models.append(("ada6",AdaBoostClassifier(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=2, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False))))
        self.models.append(("ada7",AdaBoostClassifier(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=5, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False))))
        self.models.append(("ada8",AdaBoostClassifier(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=10, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False))))

        #self.model.append(RadiusNeighborsClassifier(radius=10.0, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski'))
       
        self.models.append(("ridge1", RidgeClassifier(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, class_weight=None, solver='auto', random_state=self.random_state)))
        
        paramsGB1 = {'n_estimators': 120, 'max_depth': 3, 'subsample': 0.5,'learning_rate': 0.01, 'min_samples_leaf': 1, 'random_state': self.random_state}
        paramsGB2 = {'n_estimators': 120, 'max_depth': 6, 'subsample': 0.5,'learning_rate': 0.05, 'min_samples_leaf': 1, 'random_state': self.random_state}    
        paramsGB3 = {'n_estimators': 60, 'max_depth': 15, 'subsample': 0.5,'learning_rate': 0.01, 'min_samples_leaf': 1, 'random_state': self.random_state}
        paramsGB4 = {'n_estimators': 320, 'max_depth': 10, 'subsample': 0.5,'learning_rate': 0.005, 'min_samples_leaf': 1, 'random_state': self.random_state}
        self.models.append(("gb1",GradientBoostingClassifier(**paramsGB1)))
        self.models.append(("gb2",GradientBoostingClassifier(**paramsGB2)))
        self.models.append(("gb3",GradientBoostingClassifier(**paramsGB3)))
        self.models.append(("gb4",GradientBoostingClassifier(**paramsGB4)))

        self.models.append(("dt1",DecisionTreeClassifier(random_state=self.random_state)))
        self.models.append(("dt2",DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=3, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=self.random_state, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)))
        self.models.append(("dt3",DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=7, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=self.random_state, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)))
        self.models.append(("dt4",DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=15, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=self.random_state, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)))
        self.models.append(("dt5",DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=self.random_state, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)))


        self.models.append(("rf1",RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=2, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False)))
        self.models.append(("rf2",RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=5, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False)))
        self.models.append(("rf3",RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=10, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False)))
        
        self.models.append(("ld1",LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,solver='svd', store_covariance=False, tol=0.0001)))
       
        self.models.append(("lr1",LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=self.random_state, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)))
       
        self.models.append(("knn1",KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)))
        self.models.append(("knn2",KNeighborsClassifier(n_neighbors=10, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)))
        self.models.append(("knn3",KNeighborsClassifier(n_neighbors=15, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)))
        self.models.append(("knn4",KNeighborsClassifier(n_neighbors=20, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)))
        self.models.append(("knn5",KNeighborsClassifier(n_neighbors=50, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)))
       
        self.models.append(("nb1",GaussianNB()))
 
        self.models.append(("et1",ExtraTreesClassifier(n_estimators=50, random_state=self.random_state)))     
        self.models.append(("et2",ExtraTreesClassifier(n_estimators=100, random_state=self.random_state)))      
        self.models.append(("et3",ExtraTreesClassifier(n_estimators=200, random_state=self.random_state)))       

        self.models.append(("bag1",BaggingClassifier(base_estimator=None, n_estimators=5, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag2",BaggingClassifier(base_estimator=None, n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag3",BaggingClassifier(base_estimator=None, n_estimators=20, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag4",BaggingClassifier(base_estimator=None, n_estimators=50, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag5",BaggingClassifier(base_estimator=None, n_estimators=100, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag6",BaggingClassifier(base_estimator=None, n_estimators=150, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag7",BaggingClassifier(base_estimator=None, n_estimators=200, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag8",BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=2, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False), n_estimators=200, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag9",BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=5, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False), n_estimators=200, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag10",BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=10, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False), n_estimators=200, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))
        self.models.append(("bag11",BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=20, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,oob_score=False, random_state=self.random_state, verbose=0, warm_start=False), n_estimators=200, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=1, random_state=self.random_state, verbose=0)))

        ## add other models ...
        

    def chosen_model(self, name):
    	# initialize the available models
    	self.models_definition(self.random_state)
    	found = 0
    	for (n,i) in self.models: # n = name , i = model
    		if n == name and found == 0:
    			found = 1
    			self.selected_model = i
    			self.selected_model_name = name
    	if found == 0 :
    		# feel free to modify the model.. if another is better
    		self.selected_model = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=15, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=self.random_state, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)
    		self.selected_model_name = "dt4"
    	return


    ## to choose the best model using cross validation
    ## normally crossvalidate just the chosen model, if all_models = 1 -> crossvalidate all the models 
    def crossValidation(self, all_models = 0, k_fold = 10, random_state = 12345678):
        # cross validation
        if all_models == 1:
            print ("begin cross validation for all models")
            evaluation = []
            counter = 1
            numberOfModels = len(self.models)
            #best = ("BEST", 0, 0)
            for (name,i) in self.models:
                print (round(counter / numberOfModels,3), " is complete \t" )
                e = model_selection.cross_val_score(i, self.cv_x, self.cv_y, cv=StratifiedKFold(n_splits=self.k_fold,random_state=self.random_state,shuffle=True))
                avg = round(np.average(e),4) * 100
                std = round(np.std(e),4) * 100
                evaluation.append ((name, avg , std))
                counter = counter + 1
            evaluation.sort(key = lambda tup: tup[1], reverse = True)
            df_cv = pd.DataFrame (evaluation)
            print ("end cross validation")
            return df_cv
        else:
            e = model_selection.cross_val_score(self.selected_model, self.cv_x, self.cv_y, cv=StratifiedKFold(n_splits=k_fold,random_state=random_state,shuffle=True))
            return pd.DataFrame((self.selected_model_name, round(np.average(e),4) * 100 , round(np.std(e),4) * 100 ))
        return

    def showData(self, lines = 5, original_data = 0):
        if original_data == 1:
            print (self.original_data.head(lines))
        else :
            print(self.data.head(lines))
     

    # remove unused features, convert categorical attributes to numerical ones
    def cleanData(self):
        print ("START CLEANING")
        # re-start from the orginial data
        self.data = self.original_data
        if 'Soggetti' in self.data.columns:
        	self.data = self.data.drop('Soggetti', axis = 1)
        if 'PCneg' in self.data.columns:
        	self.data = self.data.drop('PCneg', axis = 1)
        if 'IPG' in self.data.columns:
        	self.data = self.data.drop('IPG', axis = 1)

        # convert categorical variables into numerical 
        if 'patsex' in self.data.columns:
        	self.data['patsex'] = self.data['patsex'].map({'männlich' : 1, 'weiblich' : 0})

        if 'AFclass' in self.data.columns:
        	self.data["AFclass"] = self.data["AFclass"].map({'persistierend (>7 Tage, EKV)' : 1, 'paroxysmal' : 0}) 

		# extract features
        self.features = self.data.columns[self.data.columns != self.target_name]
        self.X = self.data[self.features]
        self.target = self.data[self.target_name]
        print ("END CLEANING")
    
    # clean the test data -> first drop unused data -> make it "compliant" to the features of the dataset 
    def cleanDataTest(self, test_x):
        print ("START TEST CLEANING")
        # convert categorical variables into numerical 
        if 'patsex' in test_x.columns:
            test_x['patsex'] = test_x['patsex'].map({'männlich' : 1, 'weiblich' : 0})

        
        # drop all the columns that are not present in the training dataset
        for i in test_x.columns:
            if i not in self.features:
                test_x = test_x.drop(i, axis = 1)

        # add columns that are not present in the test set
        for i in self.features:
            if i not in test_x.columns:
                test_x[i] = np.nan

        ## REORDER the features
        test_x = test_x[self.features]
        print ("END TEST CLEANING")
        return test_x
    
    ## TO BE MODIFIED -> check the cleanData function
    '''
    def cleanDataTest(self):
        print ("START TEST CLEANING")
        self.test_set = self.test_set.drop('Soggetti', axis = 1)
        self.test_set = self.test_set.drop('PCneg', axis = 1)
        self.test_set = self.test_set.drop('IPG', axis = 1)
        # convert categorical variables into numerical 
        self.test_set['patsex'] = self.test_set['patsex'].map({'männlich' : 1, 'weiblich' : 0})
        print ("END TEST CLEANING")
    '''
    ## data -> it is the dataset we want to 'recover'
    def recoverMissing(self, data = 'trainData', imputation_strategy = 'mice'):
        print ("START RecoverMissing VALUES")
        if str(data) == 'trainData':
            x_incomplete = self.data[self.features]	
        else:
            x_incomplete = data[self.features]
        
        if imputation_strategy == 'knn':
            x_complete_a = fancyimpute.KNN(15).complete(x_incomplete)
        ## feel free to add other imputation methods 
        # ... 
        else : ## default case -> MICE impute method
            mice = fancyimpute.MICE(n_imputations=100, impute_type='col', n_nearest_columns=5)
            x_complete_a = mice.complete(x_incomplete)
        x_complete = pd.DataFrame(x_complete_a, columns = self.features)
        if str(data) == 'trainData':
            self.X = x_complete
        return x_complete
    
    def balanceDataSet(self, data = "trainData",target_name = "AFclass", balance_strategy = 'SMOTE'):
        if str(data) == "trainData":
            X = self.X
            y = self.data[self.target_name].as_matrix()
            target_name = self.target_name
        else :
            X = data[data.columns[data.columns != target_name]]
            y = data[target_name].as_matrix()
        y_new = pd.DataFrame(y)
        y_new = y_new.rename(columns = {y_new.columns[0] : target_name})
        Data_complete = pd.concat([X,y_new], axis = 1)
        if balance_strategy == 'ADASYN':
            try:
                print ("Try ADASYN")
                X_resampled, y_resampled = ADASYN().fit_sample(X, y_new)
            except:
                print ("ADASYN FAILED -> used SMOTE")
                X_resampled, y_resampled = SMOTE().fit_sample(X, y_new)

            ## feel free to add other balancing strategies
            # ...
        else : # default SMOTE
            X_resampled, y_resampled = SMOTE().fit_sample(X, y_new)



        X_final = pd.DataFrame(X_resampled, columns = self.features)
        Y_final = pd.DataFrame(y_resampled)
        Y_final = Y_final.rename(columns = {Y_final.columns[0] : self.target_name})

        Data_final = pd.concat([X_final,Y_final], axis = 1)
        if str(data) == "trainData" :
            self.X = X_final
            self.target = Y_final
            self.cv_x = X_final
            self.cv_y = Y_final
            self.data = Data_final
        return Data_final

    '''
    def recoverMissing(self, imputation_strategy = 'mice', balance_strategy = 'SMOTE'):
        print ("START RecoverMissing VALUES")
        x_incomplete = self.data[self.features]
        y = self.data[self.target_name].as_matrix()
        #y = self.data[self.target_name]
        print ("YYYY " ,y)
        
        # impute missing value with KNN strategy
        if imputation_strategy == 'knn':
            x_complete_a = fancyimpute.KNN(15).complete(x_incomplete)
        ## feel free to add other imputation methods 
        # ... 
        else : ## default case -> MICE impute method
            mice = fancyimpute.MICE(n_imputations=100, impute_type='col', n_nearest_columns=5)
            x_complete_a = mice.complete(x_incomplete)
        x_complete = pd.DataFrame(x_complete_a, columns = self.features)
        y_new = pd.DataFrame(y)
        y_new = y_new.rename(columns = {y_new.columns[0] : self.target_name})
        Data_complete = pd.concat([x_complete,y_new], axis = 1)
        
        
        ## check if the dataset is balanced -> if not balace it , by adding increasing the smallest class
        #Data_pers_size = Data_complete[Data_complete[self.target_name] == 1].shape[0] # number of rows
        #Data_paro_size = Data_complete[Data_complete[self.target_name] == 0].shape[0] # number of rows
        #print ("SIZE OF THE CLASSES :",Data_pers_size ,Data_paro_size)
        #if Data_paro_size != Data_pers_size :
        if balance_strategy == 'ADASYN':
            try:
                print ("Try ADASYN")
                X_resampled, y_resampled = ADASYN().fit_sample(x_complete, y_new)
            except:
                print ("ADASYN FAILED -> used SMOTE")
                X_resampled, y_resampled = SMOTE().fit_sample(x_complete, y_new)

            ## feel free to add other balancing strategies
            # ...
        else : # default SMOTE
            X_resampled, y_resampled = SMOTE().fit_sample(x_complete, y_new)



        X_final = pd.DataFrame(X_resampled, columns = self.features)
        Y_final = pd.DataFrame(y_resampled)
        Y_final = Y_final.rename(columns = {Y_final.columns[0] : self.target_name})

        Data_final = pd.concat([X_final,Y_final], axis = 1)

        self.X = X_final
        self.target = Y_final
        self.cv_x = X_final
        self.cv_y = Y_final

        self.data = Data_final


        print ("END RecoverMissing VALUES")
    '''
    # clean the data, recover missing values, balance the dataset
    def fixDataset(self, imputation_strategy = 'mice', balance_strategy = 'SMOTE'):
        print ("begin fixing dataset")
        self.cleanData()
        self.recoverMissing(imputation_strategy = imputation_strategy)
        self.balanceDataSet(balance_strategy = balance_strategy)
        print ("end fixing dataset")

    # train the selected model
    def train(self):
        ## use all the availble data -> we assume to know what is the best model -> otherwise use the crossvalidation function to choose a model
        print ("begin training")
        self.selected_model.fit(self.X, self.target)
        print ("end training")


    # predict using the trained model. x_test is a vector 
    # return the prediction for all values in the vector x_test, and all the other useful data (according to the selected_model used to predict)
    def predict(self, test):
        original_x = test
        x_test = self.cleanDataTest(test_x = test)
        x_test = self.recoverMissing(data = x_test)
        result = original_x
        prediction = self.selected_model.predict(x_test)
        result['prediction'] = prediction
        predict_proba = None
        predict_log_proba = None
        #decision_path = None
        #features_importance = None
        '''
        if callable(getattr(self.selected_model, "predict_proba" )):
            #predict_proba =  self.selected_model.predict_proba(x_test)
            predict_proba_df = pd.DataFrame(self.selected_model.predict_proba(x_test), columns=self.selected_model.classes_)
            result = pd.concat([result,predict_proba_df], axis = 1)
            

        if callable(getattr(self.selected_model, "predict_log_proba" )):
            #predict_proba =  self.selected_model.predict_log_proba(x_test)
            #result['predict_log_proba'] = predict_proba
            predict_log_proba_df = pd.DataFrame(self.selected_model.predict_log_proba(x_test), columns=self.selected_model.classes_)
            result = pd.concat([result,predict_log_proba_df], axis = 1)
        '''
        return pd.DataFrame(result)

    ## TO BE MODIFIED -> check the recoverMissing function
    '''
    def recoverMissingTest(self):
        print ("START RecoverMissing VALUES TEST")   
        # impute missing value with KNN strategy
        x_knn_a = fancyimpute.KNN(15).complete(self.test_set)
        x_knn = pd.DataFrame(x_knn_a, columns = self.features)
        self.test_set = x_knn
        print ("END RecoverMissing VALUES TEST")
    '''

    ## TO BE MODIFIED / REVIEWD 
    '''
    # train the models
    def train(self):
        
        self.cleanData()
        self.recoverMissing()
        
        # use all  the data for training ? is it a good idea ? check questions file
        x = self.data[self.features]
        y = self.data[self.target]
        random_state = 11223344
        print ("START TRAINING")
        self.sl_train(x,y,random_state)
        self.rf_train(x,y,random_state)
        self.knn_train(x,y)
        self.nb_train(x,y)
        self.svc_train(x,y,random_state)
        print ("END TRAINING")
    
    def sl_train(self,x,y,random_state):
        print ("start sl training")
        self.sl = LogisticRegression(C=10e10, random_state=random_state);
        self.sl.fit(x,y);
        #y_p = sl.predict(test);
        print ("end sl training")

    def rf_train(self,x,y,random_state):
        print ("start rf training")
        self.rf = RandomForestClassifier(n_estimators=40, max_depth=None, min_samples_split=2, random_state=random_state)
        self.rf.fit(x,y);
        #Y_P_bal_rf_1 = rf_cm.predict(X_bal_1);
        print ("end rf training")
            
    def knn_train(self,x,y):
        print ("start knn training")
        self.knn = neighbors.KNeighborsClassifier(20, weights='distance')
        self.knn.fit(x,y);
        #Y_P_bal_knn_1 = knn_cm.predict(X_bal_1);
        print ("end knn training")

    def nb_train(self,x,y):
        print ("start nb training")
        self.nb = naive_bayes.GaussianNB();
        self.nb.fit(x,y);
        #Y_P_bal_nb_1 = nb_cm.predict(X_bal_1);
        print ("end nb training")
    
    def svc_train (self,x,y,random_state):
        print ("start svm training")
        #self.svc = svm.SVC(kernel='linear', C = 1.0)
        self.svc = svm.SVC(kernel='rbf',C=1, probability=True, random_state=random_state)
        self.svc.fit(x,y)
        #(clf.predict([0.58,0.76]))
        print ("end svm training")

    def predict(self, x):
        print ("START PREDICTION")
        
        # check if x has null values -> if so ? impute missings ?
        self.test_set = x
        self.cleanDataTest()
        self.recoverMissingTest()
        
        sl_y_p = self.sl.predict(self.test_set)
        rf_y_p = self.rf.predict(self.test_set)
        knn_y_p = self.knn.predict(self.test_set)
        nb_y_p = self.nb.predict(self.test_set)
        svc_y_p = self.svc.predict(self.test_set)
        
        # aggregate predictions -> MODE
        prediction = pd.DataFrame({"sl": sl_y_p, "rf": rf_y_p, "knn": knn_y_p, "nb" : nb_y_p, "svc" : svc_y_p})
        prediction['MODE'] = prediction.mode(axis= 1)
        print ("END PREDICTION")
        
        return prediction['MODE']

   	'''

In [12]:
g = LocalModel(trainData, "AFclass")

begin fixing dataset
START CLEANING
END CLEANING
START RecoverMissing VALUES
[MICE] Completing matrix with shape (458, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.006
[MICE] Starting imputation round 2/110, elapsed time 0.135
[MICE] Starting imputation round 3/110, elapsed time 0.166
[MICE] Starting imputation round 4/110, elapsed time 0.196
[MICE] Starting imputation round 5/110, elapsed time 0.232
[MICE] Starting imputation round 6/110, elapsed time 0.265
[MICE] Starting imputation round 7/110, elapsed time 0.299
[MICE] Starting imputation round 8/110, elapsed time 0.334
[MICE] Starting imputation round 9/110, elapsed time 0.364
[MICE] Starting imputation round 10/110, elapsed time 0.403
[MICE] Starting imputation round 11/110, elapsed time 0.435
[MICE] Starting imputation round 12/110, elapsed time 0.474
[MICE] Starting imputation round 13/110, elapsed time 0.504
[MICE] Starting imputation round 14/110, elapsed time 0.538
[MICE] Starting imputation round 15/110, elaps

In [13]:
g.train()

begin training
end training


In [14]:
test_x_p.head()

,Soggetti,RR,QTm_old,sbjBeatConsidered,numRRaveraged,QR,QTn,QRS,IPG,PQ,PCpos,PCneg,PCdur,PCfwhm,PCarea,P_LoffEon,Pdur16mean,Pdur16median,Pdur16min,Pdur16max,Pdur16std,Pdur16iqr,Pdur_ECG_I,Pdur_ECG_II,Pdur_ECG_III,Pdur_ECG_V1,Pdur_ECG_V2,Pdur_ECG_V3,Pdur_ECG_V3R,Pdur_ECG_V4,Pdur_ECG_V4R,Pdur_ECG_V5,Pdur_ECG_V6,Pdur_ECG_V8,Pdur_ECG_V9,Pdur_ECG_aVF,Pdur_ECG_aVL,Pdur_ECG_aVR,Parea_ECG_I,Parea_ECG_II,Parea_ECG_III,Parea_ECG_V1,Parea_ECG_V2,Parea_ECG_V3,Parea_ECG_V3R,Parea_ECG_V4,Parea_ECG_V4R,Parea_ECG_V5,Parea_ECG_V6,Parea_ECG_V8,Parea_ECG_V9,Parea_ECG_aVF,Parea_ECG_aVL,Parea_ECG_aVR,Pfwhm_ECG_I,Pfwhm_ECG_II,Pfwhm_ECG_III,Pfwhm_ECG_V1,Pfwhm_ECG_V2,Pfwhm_ECG_V3,Pfwhm_ECG_V3R,Pfwhm_ECG_V4,Pfwhm_ECG_V4R,Pfwhm_ECG_V5,Pfwhm_ECG_V6,Pfwhm_ECG_V8,Pfwhm_ECG_V9,Pfwhm_ECG_aVF,Pfwhm_ECG_aVL,Pfwhm_ECG_aVR,patsex,Age
190,10_0168,1006.889706,404.0,279,272,52.0,402.615430,92.0,0,184.0,169.5479,-17.13073,126.0,66.11688,11148.290,149.0,129.0625,128.5,111.0,145.0,10.604830,14.0,138.0,130.0,131.0,140.0,111.0,127.0,125.0,129.0,111.0,125.0,141.0,128.0,114.0,128.0,142.0,145.0,3029.4220,4996.800,4485.879,2882.013,633.1925,3378.605,3228.258,2756.582,2604.802,2206.398,2768.152,1373.8070,1805.9510,4537.300,2041.478,4360.819,77.00324,63.01426,71.00903,47.00362,28.00112,60.00124,57.00391,58.00215,50.02497,53.00547,58.00011,NaN,63.00057,68.00850,40.00005,62.00015,männlich,61
75,02_0220,1012.480000,389.0,243,225,26.0,386.595126,68.0,0,213.0,267.6222,NaN,112.0,47.18302,13806.560,121.0,108.8125,111.5,84.0,119.0,9.927865,6.5,115.0,107.0,116.0,112.0,84.0,86.0,110.0,114.0,111.0,111.0,113.0,109.0,119.0,114.0,105.0,115.0,999.5247,4607.351,3722.045,3052.869,3636.9060,4898.086,5677.808,4426.514,4401.285,3449.531,2445.199,593.8314,276.9914,4182.917,1029.307,2844.739,53.40934,53.00143,53.00103,35.00375,46.00161,46.02458,43.00087,51.00281,43.04072,53.02073,56.00436,59.04374,28.25346,53.00006,55.35048,51.00027,männlich,74
96,01_2310,731.767016,319.0,390,382,25.0,372.910205,58.0,0,204.0,167.2284,NaN,135.0,64.00834,12061.860,159.0,136.0000,135.0,126.0,156.0,6.831301,8.5,140.0,140.0,135.0,130.0,131.0,135.0,131.0,135.0,133.0,135.0,156.0,126.0,132.0,135.0,140.0,142.0,3315.8400,6238.145,3341.217,3576.511,2026.5420,3629.043,2648.504,3606.622,1862.922,3265.118,3396.558,2476.2160,2303.2030,4279.942,1252.347,4756.018,92.00061,65.03931,65.00011,NaN,32.00175,47.00235,46.00005,66.00045,59.00526,70.00207,93.00021,72.00448,67.00000,61.00668,28.00005,70.01204,weiblich,85
253,12_0054,1074.082353,461.0,262,255,35.0,444.817776,68.0,0,200.0,170.7982,-117.09800,94.0,NaN,7174.958,129.0,98.6875,96.5,79.0,126.0,12.985730,20.0,104.0,79.0,115.0,83.0,98.0,86.0,105.0,89.0,126.0,95.0,85.0,110.0,103.0,95.0,112.0,94.0,765.7574,2181.231,2064.051,1561.403,3589.0870,3217.238,1872.782,3130.457,1541.022,2767.441,2056.330,1105.0800,1059.5590,1769.627,1691.996,1513.653,71.00329,36.00895,27.07151,NaN,35.64871,30.42365,51.12453,31.10153,47.00254,32.00034,NaN,NaN,48.00087,26.05479,26.07209,33.00315,männlich,78
277,13_0066,982.059041,440.0,279,271,30.0,444.000928,74.0,0,179.0,410.9566,NaN,113.0,42.66142,20959.370,121.0,111.6875,113.5,101.0,120.0,5.510218,5.5,120.0,114.0,102.0,115.0,114.0,114.0,114.0,101.0,113.0,112.0,115.0,104.0,113.0,112.0,106.0,118.0,8107.9320,6074.004,2740.810,5530.867,6553.9780,7276.250,5255.694,7087.796,4989.171,7696.956,6539.705,4018.0210,2509.8200,2818.111,4517.707,7004.745,63.01338,44.18743,27.05418,30.02472,33.14719,36.91283,31.01521,39.18646,30.02318,53.03702,63.00008,51.00736,40.02856,42.00107,53.01152,59.01987,männlich,71


In [15]:
test_x_copy = test_x_p

In [16]:
predicted_class = g.predict(test = test_x_copy)
predicted_class

START TEST CLEANING
END TEST CLEANING
START RecoverMissing VALUES
[MICE] Completing matrix with shape (226, 69)
[MICE] Starting imputation round 1/110, elapsed time 0.003
[MICE] Starting imputation round 2/110, elapsed time 0.033
[MICE] Starting imputation round 3/110, elapsed time 0.064
[MICE] Starting imputation round 4/110, elapsed time 0.097
[MICE] Starting imputation round 5/110, elapsed time 0.128
[MICE] Starting imputation round 6/110, elapsed time 0.158
[MICE] Starting imputation round 7/110, elapsed time 0.193
[MICE] Starting imputation round 8/110, elapsed time 0.229
[MICE] Starting imputation round 9/110, elapsed time 0.267
[MICE] Starting imputation round 10/110, elapsed time 0.308
[MICE] Starting imputation round 11/110, elapsed time 0.354
[MICE] Starting imputation round 12/110, elapsed time 0.395
[MICE] Starting imputation round 13/110, elapsed time 0.437
[MICE] Starting imputation round 14/110, elapsed time 0.478
[MICE] Starting imputation round 15/110, elapsed time 0.5

,Soggetti,RR,QTm_old,sbjBeatConsidered,numRRaveraged,QR,QTn,QRS,IPG,PQ,PCpos,PCneg,PCdur,PCfwhm,PCarea,P_LoffEon,Pdur16mean,Pdur16median,Pdur16min,Pdur16max,Pdur16std,Pdur16iqr,Pdur_ECG_I,Pdur_ECG_II,Pdur_ECG_III,Pdur_ECG_V1,Pdur_ECG_V2,Pdur_ECG_V3,Pdur_ECG_V3R,Pdur_ECG_V4,Pdur_ECG_V4R,Pdur_ECG_V5,Pdur_ECG_V6,Pdur_ECG_V8,Pdur_ECG_V9,Pdur_ECG_aVF,Pdur_ECG_aVL,Pdur_ECG_aVR,Parea_ECG_I,Parea_ECG_II,Parea_ECG_III,Parea_ECG_V1,Parea_ECG_V2,Parea_ECG_V3,Parea_ECG_V3R,Parea_ECG_V4,Parea_ECG_V4R,Parea_ECG_V5,Parea_ECG_V6,Parea_ECG_V8,Parea_ECG_V9,Parea_ECG_aVF,Parea_ECG_aVL,Parea_ECG_aVR,Pfwhm_ECG_I,Pfwhm_ECG_II,Pfwhm_ECG_III,Pfwhm_ECG_V1,Pfwhm_ECG_V2,Pfwhm_ECG_V3,Pfwhm_ECG_V3R,Pfwhm_ECG_V4,Pfwhm_ECG_V4R,Pfwhm_ECG_V5,Pfwhm_ECG_V6,Pfwhm_ECG_V8,Pfwhm_ECG_V9,Pfwhm_ECG_aVF,Pfwhm_ECG_aVL,Pfwhm_ECG_aVR,patsex,Age,prediction
190,10_0168,1006.889706,404.0,279,272,52.0,402.615430,92.0,0,184.0,169.547900,-17.130730,126.0,66.11688,11148.290,149.0,129.0625,128.5,111.0,145.0,10.604830,14.0,138.0,130.0,131.0,140.0,111.0,127.0,125.0,129.0,111.0,125.0,141.0,128.0,114.0,128.0,142.0,145.0,3029.4220,4996.8000,4485.8790,2882.0130,633.1925,3378.6050,3228.2580,2756.5820,2604.8020,2206.3980,2768.1520,1373.80700,1805.9510,4537.3000,2041.4780,4360.8190,77.00324,63.01426,71.009030,47.00362,28.00112,60.00124,57.00391,58.00215,50.02497,53.00547,58.00011,NaN,63.00057,68.00850,40.000050,62.00015,1,61,1
75,02_0220,1012.480000,389.0,243,225,26.0,386.595126,68.0,0,213.0,267.622200,NaN,112.0,47.18302,13806.560,121.0,108.8125,111.5,84.0,119.0,9.927865,6.5,115.0,107.0,116.0,112.0,84.0,86.0,110.0,114.0,111.0,111.0,113.0,109.0,119.0,114.0,105.0,115.0,999.5247,4607.3510,3722.0450,3052.8690,3636.9060,4898.0860,5677.8080,4426.5140,4401.2850,3449.5310,2445.1990,593.83140,276.9914,4182.9170,1029.3070,2844.7390,53.40934,53.00143,53.001030,35.00375,46.00161,46.02458,43.00087,51.00281,43.04072,53.02073,56.00436,59.04374,28.25346,53.00006,55.350480,51.00027,1,74,1
96,01_2310,731.767016,319.0,390,382,25.0,372.910205,58.0,0,204.0,167.228400,NaN,135.0,64.00834,12061.860,159.0,136.0000,135.0,126.0,156.0,6.831301,8.5,140.0,140.0,135.0,130.0,131.0,135.0,131.0,135.0,133.0,135.0,156.0,126.0,132.0,135.0,140.0,142.0,3315.8400,6238.1450,3341.2170,3576.5110,2026.5420,3629.0430,2648.5040,3606.6220,1862.9220,3265.1180,3396.5580,2476.21600,2303.2030,4279.9420,1252.3470,4756.0180,92.00061,65.03931,65.000110,NaN,32.00175,47.00235,46.00005,66.00045,59.00526,70.00207,93.00021,72.00448,67.00000,61.00668,28.000050,70.01204,0,85,1
253,12_0054,1074.082353,461.0,262,255,35.0,444.817776,68.0,0,200.0,170.798200,-117.098000,94.0,NaN,7174.958,129.0,98.6875,96.5,79.0,126.0,12.985730,20.0,104.0,79.0,115.0,83.0,98.0,86.0,105.0,89.0,126.0,95.0,85.0,110.0,103.0,95.0,112.0,94.0,765.7574,2181.2310,2064.0510,1561.4030,3589.0870,3217.2380,1872.7820,3130.4570,1541.0220,2767.4410,2056.3300,1105.08000,1059.5590,1769.6270,1691.9960,1513.6530,71.00329,36.00895,27.071510,NaN,35.64871,30.42365,51.12453,31.10153,47.00254,32.00034,NaN,NaN,48.00087,26.05479,26.072090,33.00315,1,78,0
277,13_0066,982.059041,440.0,279,271,30.0,444.000928,74.0,0,179.0,410.956600,NaN,113.0,42.66142,20959.370,121.0,111.6875,113.5,101.0,120.0,5.510218,5.5,120.0,114.0,102.0,115.0,114.0,114.0,114.0,101.0,113.0,112.0,115.0,104.0,113.0,112.0,106.0,118.0,8107.9320,6074.0040,2740.8100,5530.8670,6553.9780,7276.2500,5255.6940,7087.7960,4989.1710,7696.9560,6539.7050,4018.02100,2509.8200,2818.1110,4517.7070,7004.7450,63.01338,44.18743,27.054180,30.02472,33.14719,36.91283,31.01521,39.18646,30.02318,53.03702,63.00008,51.00736,40.02856,42.00107,53.011520,59.01987,1,71,1
305,15_0080,1206.444444,408.0,239,234,38.0,371.455249,82.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,69,1
69,01_2614,1057.005495,437.0,203,182,37.0,425.052735,NaN,0,274.0,89.123380,-11.443200,13

In [22]:
predicted = predicted_class["prediction"]
predicted

190    1
75     1
96     1
253    0
277    1
305    1
69     0
217    1
217    1
438    1
328    1
2      0
413    1
705    0
169    1
129    1
296    0
104    1
293    1
27     1
567    1
492    1
516    1
649    1
334    1
694    0
329    0
262    0
105    1
308    1
323    0
282    1
243    0
370    0
60     0
220    1
524    0
234    1
74     0
279    0
337    0
50     0
59     1
555    1
111    0
343    0
81     0
411    0
163    1
20     0
322    1
31     1
311    0
315    1
159    1
48     0
587    1
744    1
171    0
93     0
124    1
677    0
399    0
121    1
272    0
594    1
247    1
283    0
128    0
731    0
499    1
725    1
175    0
327    0
215    0
202    0
301    1
256    1
288    0
203    1
214    1
307    0
376    1
284    1
57     0
310    0
581    1
29     0
232    1
0      1
137    0
668    1
214    0
239    0
263    0
564    0
124    0
333    1
89     1
224    0
325    1
151    0
285    0
647    0
38     0
152    1
173    0
37     1
322    0
95     1
703    0
2

In [26]:
#predicted_class = g.predict(test_x_p)
predicted_y = predicted.ravel()
#test_y = test_y.map({'persistierend (>7 Tage, EKV)' : 1, 'paroxysmal' : 0}).ravel()
countPerTrue = 0
countParTrue = 0
countPerFalse = 0
countParFalse = 0
for i in range(test_y.size):
    if (test_y[i] == predicted_y[i]) and (test_y[i]== 1):
        countPerTrue += 1
    if (test_y[i] == predicted_y[i]) and (test_y[i]== 0):
        countParTrue += 1
    if (test_y[i] != predicted_y[i]) and (test_y[i]== 1):
        countParFalse += 1
    if (test_y[i] != predicted_y[i]) and (test_y[i]== 0):
        countPerFalse += 1
    #print (Y_bal_1_array[i],Y_P_bal_1[i])
print ("FOR logistic regression")
print ("result true pers : ", countPerTrue)
print ("result true paro : ", countParTrue)
print ("result false pers : ", countPerFalse)
print ("result false paro : ", countParFalse)
print ("ACCURACY : ", (countPerTrue + countParTrue)/test_y.size)
print ("PRECISION : ", countPerTrue/ (countPerTrue+countPerFalse))
print ("RECALL : ", countPerTrue/ (countPerTrue+countParFalse))

FOR logistic regression
result true pers :  52
result true paro :  56
result false pers :  61
result false paro :  57
ACCURACY :  0.4778761061946903
PRECISION :  0.46017699115044247
RECALL :  0.47706422018348627
